In [4]:
import csv
import pandas as pd
import numpy as np
import os
import arff
import re
import codecs
path_to_dataset_folder = '../DATAsets'
path_to_ready_csv = path_to_dataset_folder + '/DATA/CSVs'
path_encoded_csv = path_to_dataset_folder + '/DATA/CSVs_reverse_encoded'
target_test_path = path_to_dataset_folder + '/DATA/Target_feature_test_CSVs'
first_nan_treat_path = path_to_dataset_folder + '/DATA/1st_NaN_Treatment'

In [ ]:
x=np.array_split(df,2)
online_retail_II1=x[0]
online_retail_II2=x[1]
online_retail_II1.to_csv(first_nan_treat_path + '/online_retail_II1.csv', index = False)
online_retail_II2.to_csv(first_nan_treat_path + '/online_retail_II2.csv', index = False)

In [5]:
file = 'Train_AccountInfo%BaseCharges.csv'
df = pd.read_csv(path_to_dataset_folder + '/DATA/Target_feature_test_CSVs' + '/' + file,header = 0, sep = ',')

In [6]:
types = df.dtypes
cardi = df.apply(pd.Series.nunique)
df2=pd.DataFrame({'Types': types,
                  'Cardinality': cardi})

In [7]:
df2[0:60]

,Types,Cardinality
TotalCharges,float64,4986
DOE,object,72
ElectronicBilling,object,2
ContractType,object,3
PaymentMethod,object,4
Target,float64,1470


In [ ]:
from datetime import datetime
from datetime import date
from dateutil.parser import parse

In [ ]:
#Used to extract the year, month, day of the week, day of the year, hour in decimal form 
#(e.g. 12:30 -> 12.5, 13:15 -> 13.25) and number of day since observation begun.
col_name = 'date'
year = []
month = []
wday = []
yday = []
hour = []
tday = []
for i in range(df.shape[0]):
    d=parse(df.loc[i,col_name])
    year.append(d.timetuple().tm_year)
    month.append(d.timetuple().tm_mon)
    wday.append(d.timetuple().tm_wday)
    yday.append(d.timetuple().tm_yday)
    hour.append(d.timetuple().tm_hour + d.timetuple().tm_min/60)
    tday.append(i+1)
df=df.assign(year=year,
             #month=month,
             wday=wday,
             yday=yday,
             #hour=hour,
             tday=tday)
df.drop(col_name,axis=1,inplace=True)
df.head(10)

In [ ]:
#This is used to randomly chose the target variable from all float type features, in all datasets
#contained in "path" dir excluding the ones with their name given as an exclusion in "exclusions",
#and move it in the last column of the dataset under the feature name "Target". The new files are
#saved in the "Target_feature_test_CSVs" folder of the "../DATAsets/DATA" dir with the feature
#selected as target, added in the back of the original file name. For manual target feature
#assignment the "target_features" can be a list containing the names of the desired features.
def random_targets(path,exclusions = None, target_features = None):
    files = os.listdir(path)
    for file in np.setdiff1d(files, exclusions,assume_unique=True):
        df = pd.read_table(path + '/' + file, header = 0, sep = ',')
        #Create the list containing the candidate features for being targets.
        elig_target = []
        for i,feat in enumerate(df.dtypes):
            #Check if it's float and not constant.
            if feat == 'float64' and len(df.iloc[:,i].unique()) > 1:
                elig_target.append(df.columns.values[i])
        if target_features != None:
            elig_target = target_features
        if len(elig_target)>0:
            
            #Set how many datasets will be produced based on the number of features.
            iterations = 1
            if i+1 >= 15 and i+1 < 24:
                iterations = 2
            elif i+1 >= 24 and i+1 < 32:
                iterations = 3
            elif i+1 >= 32 and i+1 < 40:
                iterations = 4
            elif i+1 >= 40:
                iterations = 5
                
            #Create The new datasets
            already_used = []
            for iters in range(min(iterations,len(elig_target))):
                if iters == 0:  #If this is the 1st target for a particular dataset.
                    tar_int = np.random.randint(len(elig_target))
                    target_var = elig_target[tar_int]
                    already_used.append(target_var)
                else: #If this is the 2nd or 3rd target for a particular dataset.
                    #Reload untouched dataset if more targets are needed.
                    df = pd.read_table(path + '/' + file, header = 0, sep = ',')
                    while target_var in already_used:
                        tar_int = np.random.randint(len(elig_target))
                        target_var = elig_target[tar_int]
                    already_used.append(target_var)
                target_values = df.loc[:,target_var]
                df=df.assign(Target = target_values)
                df.drop(target_var,axis=1,inplace=True)
                print(file.split('.')[0] + ' --> "' + target_var + '"')
                invalid_name_chars = [':','/','<','>','*','|','"','/']
                temp_target_var = target_var
                for char in invalid_name_chars:
                    temp_target_var = temp_target_var.replace(char,'-')
                df.to_csv(target_test_path + '/' + file.split('.')[0] + '%' + temp_target_var +
                          '.csv',index= False)
                
        else:
            print('File: ' + file + " couldn't have a target variable assigned!")
            continue

In [ ]:
random_targets(first_nan_treat_path)

In [ ]:
#Used for manual dataset generation for specific feature(s) as target(s) or for re-generating
#datasets from some SELECTED csv files.
#Move the datasets' csv in folder "manual_targets" and then specify the names of the features
#to be assigned as the new targets in the following manner; 
#random_targets(manual_path, target_features = ['foo','fee',...]).

manual_path = path_to_dataset_folder + '/manual_targets'
random_targets(manual_path, target_features = ['13:Meteo_Exterior_Crepusculo','17:Meteo_Exterior_Sol_Sud'])

In [ ]:
def linesWithNan(df):
    nansPerLine = df.isna().sum(axis=1)
    lines_with_nan = 0
    for l in nansPerLine:
        if l>0:
            lines_with_nan +=1
    return lines_with_nan

In [ ]:
#Used to perform the 1st NaN treatment of the datasets. Any Feature with more than 20% missing
#values is discarded. After all such features have been discarded if the lines with at least one
#missing vallue are less than 10% of the total remaining lines, they are droped. The resulting
#datasets are saved in "1st_NaN_Treatment" folder of the "../DATAsets/DATA" dir.
def first_nan_treatment(path,exclusions = None, feature_threshold = 0.20,
                        lines_with_nan_threshold = 0.1, visual_info = False):
    files = os.listdir(path)
    for f in np.setdiff1d(files, exclusions,assume_unique=True):
        df = pd.read_table(path + '/' + f, header = 0, sep = ',')
        lines = df.shape[0]
        features = df.shape[1]
                
        #Nans on Feature basis
        nanPerFeature = df.isna().sum(axis=0)
        indecies_to_drop = []
        for i,npf in enumerate(nanPerFeature):
            if npf / lines >= feature_threshold:
                indecies_to_drop.append(i)
        df.drop(df.columns[indecies_to_drop],axis=1,inplace=True)
        
        #Nans on Line basis
        nansPerLine = df.isna().sum(axis=1)
        lines_with_nan = linesWithNan(df)
        if (lines_with_nan / lines) <= lines_with_nan_threshold:
            df.dropna(axis = 0, how='any',inplace=True)
        new_lines_with_nan = linesWithNan(df)
        if visual_info:
            if features != df.shape[1] or lines != df.shape[0] or new_lines_with_nan/df.shape[0] >=lines_with_nan_threshold:
                print('File:' + f)
                print('Lines:' + str(lines) + ' --> ' +str(df.shape[0]))
                print('Features: ' +str(features) + ' --> '+str(df.shape[1]))
                print('Lines with NaNs: ' + str(lines_with_nan) + ' ('+
                      str(round(lines_with_nan / lines * 10000)/100) + '%) --> ' +
                      str(new_lines_with_nan)+ '('+ str(round(new_lines_with_nan /
                                                              df.shape[0] * 10000)/100) + '%)')
                print('NaNs per Feature:\n' + str(df.isna().sum(axis=0)))
                print('---------------------------------------------')
        df.to_csv(first_nan_treat_path + '/' + f,index= False)

In [ ]:
first_nan_treatment(path_encoded_csv, visual_info=True)
unusedFiles = ['BlogFeedbackData_train.csv','core_dataset.csv','household_power_consumption.csv',
              'incident_event_log.csv','job-prestige.csv','Train_Demographics.csv']
exclusions = os.listdir(path_encoded_csv)
exclusions.extend(unusedFiles)
first_nan_treatment(path_to_ready_csv,exclusions,visual_info=True)

In [ ]:
#Check all datasets after 1st nan treatment and drop any constant columns,
#while preserving the naming of the remaining columns.
for file in os.listdir(first_nan_treat_path):
    df = pd.read_table(first_nan_treat_path + '/'+ file,sep=',')
    L = df.shape[1]
    drops = []
    i=0
    for feat in df.columns:
        if len(df.loc[:,feat].unique()) == 1:
            if i == 0:
                print('Processing file: ' + file)
                print(df.head())
            print('Found one! It is column: ' + feat)
            print('Unique value = ', df.loc[0,feat])
            drops.append(feat)
            L -=1
            i+=1
    flag = False
    if df.columns[0] == '0':
        flag = True
    df.drop(drops, axis = 1, inplace = True)
    if flag:
        df.columns = range(L)
    if len(drops)>0:
        print(df.head())
    df.to_csv(first_nan_treat_path + '/'+ file,sep=',',index= False)

In [ ]:
#Used for printing NaN info for the datasets contained in dir "first_nan_treat_path"
for f in os.listdir(target_test_path):
    df =  pd.read_table(target_test_path + '/'+ f,sep=',')
    lines = df.shape[0]
    nans = df.isnull().sum(axis=1)
    nanPerFeature = df.isna().sum(axis=0)
    nanFeatureFlag = None
    indecies = []
    for i,npf in enumerate(nanPerFeature):
        if npf / lines !=0:
            nanFeatureFlag = True
            indecies.append(i)
    lines_with_nan = 0
    for l in nans:
        if l>0:
            lines_with_nan +=1
    if nanFeatureFlag:
        print('File:' + f)
        print('Lines:' + str(df.shape[0]))
        print('Features: ' +str(df.shape[1]))
        print('Lines with NaNs: ' + str(lines_with_nan) + ' ('+ 
              str(round(lines_with_nan / lines * 10000)/100) + '%)')
        print('NaNs per Feature:\n ^^^^^^^^^^^^^\n' + str(nanPerFeature[indecies])+
             '\n^^^percentages^^^')
        for i in indecies:
            print(str(round(nanPerFeature[i]/lines*10000)/100)+'%')
        print('---------------------------------------------')

In [ ]:
#Used to drop all lines in every dataset in dir '/DATAsets/DATA/Target_feature_test_CSVs',
#with NaN in the response variable.
for file in os.listdir(target_test_path):
    df = pd.read_table(target_test_path + '/' + file, header = 0, sep = ',')
    if df[df.iloc[:,-1].isnull()].index.values != []:
        print(file)
        print(df[df.iloc[:,-1].isnull()].index.values)
        df.drop(df[df.iloc[:,-1].isnull()].index, axis = 0, inplace = True)
        df.reset_index(inplace= True)
        df.to_csv(target_test_path + '/' + file,index = False)